<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/HandPD/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Data Science
import re
import csv
import json
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

# SVM
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold


Here's the plan. 

To train the model, we're going to combine a BUNCH of data. Data from Meander && Spiral from Old Hand PD and New Hand PD. Just to see how it goes. Then we'll try more detailed extraction.

Also, try only on new with age?

Run simple SVM based on output of neural network model and age and some other stats :)

CAN POSSIBLY: just extract 5 PD patients and 5 control patients for testing from new hand pd set and just train on everything else. That should be fine...... Then ez to test, just pass those in and evaluate results per patient, perhaps reach 90% accuracy with this 

We also do cross validation/validation set? then.

also, hyperparameter tuning? Can get above 90% accuracy maybe with that!

Discuss more with TT

###**Normalization**

fi' = (fi - avg)/std

In [3]:
# normalization with the paper's method (formula above)
def feature_normalization(df):
  avg_dev = df.mad(axis = 0)
  std_dev = df.std(axis = 0)

  df = df.sub(avg_dev)
  df = df.divide(std_dev)

  return df

### Extracting from Old Hand PD

In [4]:
spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/Spiral_HandPD.txt', delimiter=' ', index_col=0, names=['Image', 'Label', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])

In [5]:
spiral_df.shape

(368, 10)

In [6]:
spiral_df

,Label,RMS,Max_dist,Min_dist,SD,MRT,Max_ET,Min_ET,SD_ET,HT_ET_Diff
Image,,,,,,,,,,
1,1,3521.258301,6247.052734,30801.99219,0.014133,26.785328,176.600113,0.002130,1781.795898,0.250000
2,1,4098.876465,6032.535156,34369.70313,0.022838,26.529615,168.352737,0.084960,1443.217529,0.273585
3,1,3854.601807,6453.114746,34709.44531,0.000251,23.670755,180.898300,0.009303,1621.750000,0.256329
4,1,4069.221924,6844.231445,32181.26367,0.000168,23.456329,179.116043,0.021419,1454.390137,0.249221
5,1,4104.271973,6949.925293,36444.95313,0.004731,22.488258,188.256210,0.000000,1553.536499,0.214511
...,...,...,...,...,...,...,...,...,...,...
364,2,5593.215820,7997.703613,36371.22656,0.000190,26.964298,179.071930,0.059952,1885.214478,0.198653
365,2,7986.296387,7497.133789,29909.39063,2.613612,22.278151,203.759109,0.033372,1667.547974,0.241379
366,2,4652.542969,7275.474121,39475.41406,0.240567,21.622019,198.719940,0.000000,1469.676147,0.300000


In [7]:
spiral_df['Label'].value_counts()

2    296
1     72
Name: Label, dtype: int64

In [8]:
spiral_df.dtypes

Label           int64
RMS           float64
Max_dist      float64
Min_dist      float64
SD            float64
MRT           float64
Max_ET        float64
Min_ET        float64
SD_ET         float64
HT_ET_Diff    float64
dtype: object

In [9]:
# extracting necessary columns from spiral df
X_spiral = spiral_df[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]
y_spiral = spiral_df[['Label']]

In [10]:
meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/Meander_HandPD.txt', delimiter=' ', index_col=0, names=['Image', 'Label', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])

In [11]:
meander_df.shape

(368, 10)

In [12]:
spiral_df['Label'].value_counts()

2    296
1     72
Name: Label, dtype: int64

In [13]:
# choosing all patients from meander_df
X_meander = meander_df[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]

y_meander = meander_df[['Label']]

In [14]:
X_spiral = feature_normalization(X_spiral)
X_meander = feature_normalization(X_meander)

### Extracting from New HandPD

Don't run more than once.

In [15]:
new_spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewSpiral.csv', skiprows = 1, index_col = None, names=['ID_EXAM', 'Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS', 'AGE', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']) # skiprows to remove first row of old columns names

In [16]:
new_meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewMeander.csv', index_col = None, skiprows =1, names=['ID_EXAM', 'Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS', 'AGE', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])

In [17]:
colnames = list(new_spiral_df.columns)
colnames

['ID_EXAM',
 'Image',
 'ID_PATIENT',
 'Label',
 'GENDER',
 'HANDEDNESS',
 'AGE',
 'RMS',
 'Max_dist',
 'Min_dist',
 'SD',
 'MRT',
 'Max_ET',
 'Min_ET',
 'SD_ET',
 'HT_ET_Diff']

In [18]:
slice_cols = colnames[7:]
for i in slice_cols:
  new_spiral_df[i] = feature_normalization(new_spiral_df[i])
  new_meander_df[i] = feature_normalization(new_meander_df[i])

In [19]:
X_new_all = pd.concat([new_spiral_df, new_meander_df])
y_new_all = pd.concat([new_spiral_df["Label"], new_meander_df["Label"]])

In [20]:
np.unique(X_new_all['ID_EXAM'])

array(['H1', 'H10', 'H11', 'H12', 'H13', 'H14', 'H16', 'H17', 'H18',
       'H19', 'H2', 'H20', 'H21', 'H22', 'H23', 'H24', 'H26', 'H27',
       'H28', 'H29', 'H3', 'H30', 'H31', 'H32', 'H33', 'H34', 'H35',
       'H36', 'H37', 'H38', 'H4', 'H5', 'H7', 'H8', 'H9', 'P1', 'P10',
       'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19',
       'P2', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27',
       'P28', 'P29', 'P3', 'P30', 'P31', 'P32', 'P4', 'P6', 'P7', 'P8',
       'P9'], dtype=object)

We need 12 PD and 7 healthy. Some IDs are not completely accurate (just for a single patient, so it might be better to just choose)

We'll use IDs: H1, H10, H11, H12, H13, H14, H16 & P1, P10, P11, P12, P13, P14, P15, P16, P17, P18, P19, P2

In [21]:
X_test = pd.DataFrame(columns=['ID_EXAM', 'Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS', 'AGE', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])
X_new = X_new_all.copy()

test_ids = ['H1', 'H10', 'H11', 'H12', 'H13', 'H14', 'H16', 'P1', 'P2', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19']
for id in test_ids:
  rows = X_new_all[X_new_all['ID_EXAM'] == id]
  drop_idx = rows.index.values
  X_test = pd.concat([X_test, rows])
  X_new = X_new.drop(drop_idx, axis=0)

y_test = X_test[['Label']]

In [22]:
X_new.columns

Index(['ID_EXAM', 'Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS',
       'AGE', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET',
       'SD_ET', 'HT_ET_Diff'],
      dtype='object')

In [23]:
# choosing only control from new
X_new = X_new[X_new["Label"] == 1]
y_new = X_new[X_new['Label'] == 1]

X_new = X_new[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]
y_new = y_new[['Label']]

In [24]:
y_new.value_counts()

Label
1        224
dtype: int64

In [25]:
y_test.value_counts()

Label
2        96
1        56
dtype: int64

Great!

##**Preprocessing**

In [26]:
# concatenation

X = pd.concat([X_spiral, X_meander, X_new])
y = pd.concat([y_spiral, y_meander, y_new])

###Train-Test-Split

In [35]:
# In our train-test-split, we just want to define everything
X_train = X
y_train = y

X_test = X_test
y_test = y_test.astype('float32') -1 # subtract 1

In [37]:
X_train = X_train.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)

In [38]:
X_test = X_test.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

###**Quick Check**

In [39]:
len(y_train), len(y_test)

(960, 152)

In [40]:
y_train

,Label
0,1
1,1
2,1
3,1
4,1
...,...
955,1
956,1
957,1
958,1


In [41]:
y_test.value_counts()

Label
1.0      96
0.0      56
dtype: int64

In [42]:
y_test

,Label
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
147,1.0
148,1.0
149,1.0
150,1.0


In [43]:
y_train.value_counts()

Label
2        592
1        368
dtype: int64

##**SVM Implementation**

###**Training**

In [44]:
clf = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced', C = 10)
clf.fit(X_train, y_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

To note, it may be useful to look at the weights for this model (see which features are most valuable), but these are only relevant for a linear kernel. RBF kernel does not have relevant/interpretable weights. And of course, linear kernel is not very helpful and good in predicting (51% accuracy). The features are not linearly separable.

In [45]:
X_run = X_test[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET','SD_ET', 'HT_ET_Diff']]

In [46]:
y_pred = clf.predict(X_run)
y_proba = clf.predict_proba(X_run)

In [47]:
y_pred = pd.Series(y_pred)

##**Results**

In [48]:
y_test.dtypes

Label    float32
dtype: object

In [49]:
clf.score(X_run, y_test)

0.631578947368421

In [50]:
threshold_pred = (clf.predict_proba(X_test[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET','SD_ET', 'HT_ET_Diff']])[:,1] >= 0.5).astype(int) + 1
threshold_acc = accuracy_score(y_test.astype('float32'), threshold_pred)

In [51]:
threshold_acc

0.631578947368421

In [52]:
y_test.dtypes

Label    float32
dtype: object

In [53]:
target_names = ['Control', 'PD']
results = classification_report(y_test, y_pred, target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = confusion_matrix(y_test, y_pred)

In [54]:
results

,precision,recall,f1-score,support
Control,0.000000,0.000000,0.000000,56.000000
PD,0.631579,1.000000,0.774194,96.000000
accuracy,0.631579,0.631579,0.631579,0.631579
macro avg,0.315789,0.500000,0.387097,152.000000
weighted avg,0.398892,0.631579,0.488964,152.000000


In [55]:
conf_mat

array([[ 0, 56],
       [ 0, 96]])

* Consider probability threshold

In [56]:
TN, FP, FN, TP = conf_mat.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("TP: ", TP)
print("TN: ", TN)
print("FP: ", FP)
print("FN: ", FN)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)

TP:  96
TN:  0
FP:  56
FN:  0
Sensitivity:  1.0
Specificity:  0.0
NPV:  nan
PPV:  0.631578947368421


Can we somehow get the weights for the model to learn which features are considered most useful?

## **Log Reg Implementation**

In [57]:
clf = LogisticRegression(penalty='l1', C = 1.0, class_weight=None, random_state = None, solver = 'liblinear')
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [59]:
clf.score(X_run, y_test)

0.631578947368421

In [60]:
target_names = ['Control', 'PD']
results = classification_report(y_test, y_pred, target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = confusion_matrix(y_test, y_pred)

In [62]:
conf_mat

array([[ 0, 56],
       [ 0, 96]])

## **Cross-Validation**

In [63]:
scores_df = pd.DataFrame(columns = ['acc', 'c', 'cw', 'k'])

for i in range(10):
  train_labels = y_train.to_numpy() # train is statified so stratifying on train_labels should be stratified to whole set
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = i, test_size = 0.1, stratify = train_labels)
  
  X_val = X_val.reset_index(drop = True)
  y_val = y_val.reset_index(drop = True)

  for c in [0.01, 0.1, 1, 10, 100]:
    for cw in ['balanced', None]:
      for k in ['rbf','linear','poly']:
        clf = SVC(kernel = k, probability = True, class_weight = cw, C = c)
        clf.fit(X_train, y_train)

        thresholds = list(np.arange(0, 1, 0.01))
        best_acc = 0.0
        best_threshold = 0.0

        # tuning for best threshold
        for thresh in thresholds:
          y_pred = (clf.predict_proba(X_val)[:,1] >= thresh).astype(int) + 1
          curr_acc = accuracy_score(y_val, y_pred)
          if (curr_acc >= best_acc):
            best_acc = curr_acc
            best_threshold = thresh

        row_df = pd.DataFrame([[best_acc, best_threshold, c, cw, k]], columns = ['acc', 'thresh', 'c', 'cw', 'k'])
    
        scores_df = pd.concat([scores_df, row_df], ignore_index = True, axis=0)
  print (f'fold number {i+1} completed, score number {scores_df.shape[0]}')

fold number 1 completed, score number 30
fold number 2 completed, score number 60
fold number 3 completed, score number 90
fold number 4 completed, score number 120
fold number 5 completed, score number 150
fold number 6 completed, score number 180
fold number 7 completed, score number 210
fold number 8 completed, score number 240
fold number 9 completed, score number 270
fold number 10 completed, score number 300


In [64]:
scores_df['acc'].idxmax()

297

In [74]:
scores_df.loc[297]

acc       0.972973
c              100
cw            None
k              rbf
thresh        0.73
Name: 297, dtype: object

In [72]:
best_avg_acc = 0.0
best_avg_hyps = [0.0, 0.0, 0.0]
for c in [0.01, 0.1, 1, 10, 100]:
  for cw in ['balanced', None]:
    for k in ['rbf','linear','poly']:
      filtered = scores_df[(scores_df["cw"] == cw) & (scores_df['c'] == c) & (scores_df["k"] == k)]
      avg_acc = filtered["acc"].mean()
      if (avg_acc >= best_avg_acc):
        best_avg_acc = avg_acc
        best_avg_hyps[0] = c
        best_avg_hyps[1] = cw
        best_avg_hyps[2] = k
print("best_avg_acc: ", best_avg_acc)
print(best_avg_hyps)

best_avg_acc:  0.854209043988537
[100, 'balanced', 'rbf']


In [75]:
scores_df[(scores_df["cw"] == 'balanced') & (scores_df['c'] == 100) & (scores_df["k"] == 'rbf')]

,acc,c,cw,k,thresh
24,0.895833,100.0,balanced,rbf,0.51
54,0.885057,100.0,balanced,rbf,0.41
84,0.871795,100.0,balanced,rbf,0.62
114,0.814286,100.0,balanced,rbf,0.61
144,0.888889,100.0,balanced,rbf,0.58
174,0.824561,100.0,balanced,rbf,0.60
204,0.803922,100.0,balanced,rbf,0.70
234,0.826087,100.0,balanced,rbf,0.60
264,0.785714,100.0,balanced,rbf,0.61
294,0.945946,100.0,balanced,rbf,0.69


In [76]:
clf = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced', C = 100)
clf.fit(X_train, y_train)

SVC(C=100, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [71]:
thresholds = list(np.arange(0, 1, 0.01))

best_thresh_acc = 0.0
best_thresh = 0.0

for thresh in thresholds:
  threshold_pred = (clf.predict_proba(X_run)[:,1] >= thresh).astype(int) + 1
  threshold_acc = accuracy_score(y_test, threshold_pred)

  if threshold_acc >= best_thresh_acc:
    best_thresh_acc = threshold_acc
    best_thresh = thresh

print(best_thresh, best_thresh_acc)

0.99 0.631578947368421


In [81]:
threshold_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Question mark, what's happening above even? Validation is very similar, but just completely overfits on test set?